## Assignment 6.2
Author: Rex Gayas
Date: 20 January 2024
Modified By: N/A
Description: Aggregating Pandas dataframes and delving into time series analysis

##### Solution to Exercise 1

In [37]:
import pandas as pd

# Load the earthquakes data
earthquakes_path = "D:\\ALPHA\\Dynamic Folder\\Bellevue\\Winter 2023\\Data Wrangling\\Week 6\\earthquakes.csv"
earthquakes_df = pd.read_csv(earthquakes_path)

# Filter for earthquakes in Japan with a magnitude of 4.9 or greater of 'mb' type
japan_earthquakes = earthquakes_df[(earthquakes_df['place'].str.contains('Japan')) &
                                   (earthquakes_df['magType'] == 'mb') &
                                   (earthquakes_df['mag'] >= 4.9)]

# Display the filtered DataFrame
print(japan_earthquakes)


      mag magType           time                         place  tsunami  \
1563  4.9      mb  1538977532250  293km ESE of Iwo Jima, Japan        0   
2576  5.4      mb  1538697528010    37km E of Tomakomai, Japan        0   
3072  4.9      mb  1538579732490     15km ENE of Hasaki, Japan        0   
3632  4.9      mb  1538450871260    53km ESE of Hitachi, Japan        0   

     parsed_place  
1563        Japan  
2576        Japan  
3072        Japan  
3632        Japan  


##### Solution to Exercise 2

In [38]:
# Create bins for earthquake magnitude
bins = range(int(earthquakes_df['mag'].min()), int(earthquakes_df['mag'].max()) + 2) # +2 to include the last bin
ml_earthquakes = earthquakes_df[earthquakes_df['magType'] == 'ml']

# Use pd.cut to bin the data and value_counts to count the occurrences
magnitude_bins = pd.cut(ml_earthquakes['mag'], bins=bins, include_lowest=True, right=False)
bin_counts = magnitude_bins.value_counts().sort_index()

# Display the count of earthquakes in each bin
print(bin_counts)


mag
[-1, 0)     446
[0, 1)     2072
[1, 2)     3126
[2, 3)      985
[3, 4)      153
[4, 5)        6
[5, 6)        2
[6, 7)        0
[7, 8)        0
Name: count, dtype: int64


##### Solution to Exercise 3

In [39]:
# Load the FAANG data
faang_path = "D:\\ALPHA\\Dynamic Folder\\Bellevue\\Winter 2023\\Data Wrangling\\Week 6\\faang.csv"
faang_df = pd.read_csv(faang_path, parse_dates=['date'], index_col='date')

# Resample to monthly frequency and perform aggregations
monthly_resampled = faang_df.groupby('ticker').resample('M').agg({
    'open': 'mean',       # Mean of the opening price
    'high': 'max',        # Maximum of the high price
    'low': 'min',         # Minimum of the low price
    'close': 'mean',      # Mean of the closing price
    'volume': 'sum',      # Sum of the volume traded
})

# Reset index to flatten the DataFrame
monthly_resampled = monthly_resampled.reset_index()

# Display the resampled DataFrame
print(monthly_resampled)


   ticker       date         open         high          low        close  \
0    AAPL 2018-01-31    43.505357    45.025002    41.174999    43.501309   
1    AAPL 2018-02-28    41.819079    45.154999    37.560001    41.909737   
2    AAPL 2018-03-31    43.761786    45.875000    41.235001    43.624048   
3    AAPL 2018-04-30    42.441310    44.735001    40.157501    42.458572   
4    AAPL 2018-05-31    46.239091    47.592499    41.317501    46.384205   
5    AAPL 2018-06-30    47.180119    48.549999    45.182499    47.155357   
6    AAPL 2018-07-31    47.549048    48.990002    45.855000    47.577857   
7    AAPL 2018-08-31    53.121739    57.217499    49.327499    53.336522   
8    AAPL 2018-09-30    55.582763    57.417500    53.825001    55.518421   
9    AAPL 2018-10-31    55.300000    58.367500    51.522499    55.211413   
10   AAPL 2018-11-30    47.954881    55.590000    42.564999    47.808929   
11   AAPL 2018-12-31    41.310789    46.235001    36.647499    41.066579   
12   AMZN 20

##### Solution to Exercise 4

In [40]:
# Crosstabulation between 'tsunami' and 'magType' with max magnitude
crosstab_result = pd.crosstab(index=earthquakes_df['tsunami'], columns=earthquakes_df['magType'],
                              values=earthquakes_df['mag'], aggfunc='max').fillna(0)

# Display the crosstab result
print(crosstab_result)


magType   mb  mb_lg    md   mh   ml  ms_20    mw  mwb  mwr  mww
tsunami                                                        
0        5.6    3.5  4.11  1.1  4.2    0.0  3.83  5.8  4.8  6.0
1        6.1    0.0  0.00  0.0  5.1    5.7  4.41  0.0  0.0  7.5


###### Solution to Exercise 5

In [41]:
# Rolling 60-day aggregation of OHLC data by ticker for FAANG data
rolling_aggregations = faang_df.groupby('ticker').rolling(window='60D').agg({
    'open': 'mean',       # Mean of the opening price
    'high': 'max',        # Maximum of the high price
    'low': 'min',         # Minimum of the low price
    'close': 'mean',      # Mean of the closing price
    'volume': 'sum',      # Sum of the volume traded
}).dropna().reset_index()

# Display the rolling aggregations
print(rolling_aggregations)


     ticker       date        open        high         low       close  \
0      AAPL 2018-01-02   42.540001   43.075001   42.314999   43.064999   
1      AAPL 2018-01-03   42.836250   43.637501   42.314999   43.061249   
2      AAPL 2018-01-04   42.935833   43.637501   42.314999   43.126666   
3      AAPL 2018-01-05   43.041875   43.842499   42.314999   43.282499   
4      AAPL 2018-01-08   43.151000   43.902500   42.314999   43.343500   
...     ...        ...         ...         ...         ...         ...   
1250   NFLX 2018-12-24  283.509251  332.049988  233.679993  281.931750   
1251   NFLX 2018-12-26  281.844501  332.049988  231.229996  280.777750   
1252   NFLX 2018-12-27  281.070489  332.049988  231.229996  280.162927   
1253   NFLX 2018-12-28  279.916342  332.049988  231.229996  279.461464   
1254   NFLX 2018-12-31  278.430770  332.049988  231.229996  277.451539   

           volume  
0     102223600.0  
1     220295200.0  
2     310033600.0  
3     404673600.0  
4     48694

##### Solution to Exercise 6

In [42]:
# Create a pivot table with the ticker as rows and the averages of OHLC and volume
faang_pivot = faang_df.pivot_table(
    index='ticker', 
    values=['open', 'high', 'low', 'close', 'volume'], 
    aggfunc='mean'  # Averages of the specified columns
)

# Display the pivot table
print(faang_pivot)


              close         high          low         open        volume
ticker                                                                  
AAPL      47.263357    47.748526    46.795877    47.277859  1.360803e+08
AMZN    1641.726176  1662.839839  1619.840519  1644.072709  5.648994e+06
FB       171.510956   173.613347   169.303148   171.472948  2.765860e+07
GOOG    1113.225134  1125.777606  1101.001658  1113.554101  1.741965e+06
NFLX     319.290319   325.219322   313.187330   319.620558  1.146962e+07


##### Solution to Exercise 7

In [43]:
import pandas as pd
import numpy as np

# Load the FAANG data
faang_path = "D:\\ALPHA\\Dynamic Folder\\Bellevue\\Winter 2023\\Data Wrangling\\Week 6\\faang.csv"
faang_df = pd.read_csv(faang_path, parse_dates=['date'], index_col='date')

# Filter for Amazon's data in Q4 2018
amzn_q4_data = faang_df[(faang_df['ticker'] == 'AMZN') & 
                        (faang_df.index >= '2018-10-01') & 
                        (faang_df.index <= '2018-12-31')]

# Function to calculate Z-score
def calculate_z_score(column):
    return (column - column.mean()) / column.std()

# Calculate Z-scores using apply() on numeric columns
amzn_q4_z_scores = amzn_q4_data.select_dtypes(include=[np.number]).apply(calculate_z_score)

# Display the Z-scores
print(amzn_q4_z_scores)



                high       low      open     close    volume
date                                                        
2018-10-01  2.368006  2.502113  2.337813  2.385848 -1.630411
2018-10-02  2.227302  2.247433  2.190795  2.155037 -0.861879
2018-10-03  2.058955  2.139987  2.068570  2.025489 -0.920345
2018-10-04  1.819474  1.781561  1.850048  1.722816 -0.126582
2018-10-05  1.628173  1.554416  1.642819  1.584748 -0.298771
...              ...       ...       ...       ...       ...
2018-12-24 -2.159820 -2.187566 -2.179582 -2.226185 -0.141238
2018-12-26 -1.611714 -1.810493 -2.026617 -1.339674  1.123063
2018-12-27 -1.641276 -1.626703 -1.456521 -1.404343  0.849827
2018-12-28 -1.325261 -1.231588 -1.328549 -1.289951  0.496102
2018-12-31 -1.273456 -0.975763 -1.078283 -1.122691 -0.246405

[63 rows x 5 columns]


##### Solution to Exercise 8

In [44]:
import pandas as pd

# a) Create a DataFrame with event descriptions for FB
events = {
    'ticker': 'FB',
    'date': pd.to_datetime(['2018-07-25', '2018-03-19', '2018-03-20']),
    'event': [
        'Disappointing user growth announced after close.',
        'Cambridge Analytica story',
        'FTC investigation'
    ]
}
events_df = pd.DataFrame(events).set_index(['date', 'ticker'])

# Reset index for faang_df to ensure the merge operation can be performed
faang_df.reset_index(inplace=True)

# Ensure 'date' is of datetime type
faang_df['date'] = pd.to_datetime(faang_df['date'])

# Set the new multi-index for faang_df
faang_df.set_index(['date', 'ticker'], inplace=True)

# c) Merge with the FAANG data using an outer join
merged_data = faang_df.merge(events_df, how='outer', left_index=True, right_index=True)

# Display the merged DataFrame
print(merged_data)


                          high          low         open        close  \
date       ticker                                                       
2018-01-02 AAPL      43.075001    42.314999    42.540001    43.064999   
           AMZN    1190.000000  1170.510010  1172.000000  1189.010010   
           FB       181.580002   177.550003   177.679993   181.419998   
           GOOG    1066.939941  1045.229980  1048.339966  1065.000000   
           NFLX     201.649994   195.419998   196.100006   201.070007   
...                        ...          ...          ...          ...   
2018-12-31 AAPL      39.840000    39.119999    39.632500    39.435001   
           AMZN    1520.760010  1487.000000  1510.800049  1501.969971   
           FB       134.639999   129.949997   134.449997   131.089996   
           GOOG    1052.699951  1023.590027  1050.959961  1035.609985   
           NFLX     270.100006   260.000000   260.160004   267.660004   

                        volume event  
date       

##### Solution to Exercise 9

In [45]:
import pandas as pd

# Function to divide by the first row in the group
def index_to_base(x):
    return x / x.iloc[0]

# Use transform() to apply the indexing function to each group
faang_indexed = faang_df.groupby('ticker').transform(index_to_base)

# Display the transformed DataFrame
print(faang_indexed)


                       high       low      open     close    volume
date       ticker                                                  
2018-01-02 FB      1.000000  1.000000  1.000000  1.000000  1.000000
2018-01-03 FB      1.017623  1.021290  1.023638  1.017914  0.930294
2018-01-04 FB      1.025498  1.036891  1.040635  1.016040  0.764708
2018-01-05 FB      1.029298  1.041566  1.044518  1.029931  0.747828
2018-01-08 FB      1.040313  1.049451  1.053579  1.037813  0.991340
...                     ...       ...       ...       ...       ...
2018-12-24 GOOG    0.940578  0.928131  0.928993  0.916638  1.284987
2018-12-26 GOOG    0.974750  0.940463  0.943406  0.976019  1.917663
2018-12-27 GOOG    0.978396  0.953857  0.970248  0.980169  1.704751
2018-12-28 GOOG    0.989334  0.988395  1.001221  0.973784  1.143180
2018-12-31 GOOG    0.986653  0.979296  1.002499  0.972404  1.206610

[1255 rows x 5 columns]
